In [33]:
# RCS STATISTICAL FEATURE EXTRACTION FOR SPACE TARGET RECOGNITION BASED ON BI-LSTM Yanbing Wang, Bo Long and Feng Wang*
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def db(data):
    return 10*np.log10(data)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

def extract_features(data, window_size, step_size):
    features = []
    for start in range(0, len(data) - window_size + 1, step_size):
        window = data[start:start + window_size]
        feature = [
            np.mean(window),
            np.max(window),
            np.min(window),
            np.std(window),
            np.ptp(window),
            np.median(window),
            pd.Series(window).skew(),
            pd.Series(window).kurt(),
            np.sqrt(np.mean(window**2)),
            np.mean(np.abs(window)),
            np.sum(np.abs(np.diff(window)))
        ]
        features.append(feature)
    return np.array(features)

cuda


In [34]:
# Load the data
snr = 20

train_data = pd.read_csv(f'train_{snr}db.csv')
test_data = pd.read_csv(f'test_{snr}db.csv')

class TimeSeriesDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.labels = data.iloc[:, -1].values
        self.features = data.iloc[:, :-1].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.features[idx]
        x = extract_features(x, window_size=400, step_size=100)
        y = self.labels[idx]
        return torch.tensor(x, dtype=torch.float32).to(device), torch.tensor(y, dtype=torch.long).to(device)

train_dataset = TimeSeriesDataset(train_data)
test_dataset = TimeSeriesDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [35]:
print(f'extract feature data with shape {train_dataset.__getitem__(0)[0].cpu().numpy().shape}')
for data,label in train_loader:
    print(data.shape, label.shape)
    break

extract feature data with shape (17, 11)
torch.Size([128, 17, 11]) torch.Size([128])


In [36]:
class BiLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        # self.leakyRelu = nn.LeakyReLU(0.3)
        # self.dropout = nn.Dropout(0.5)
        self.fc_1 = nn.Linear(hidden_size , num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :self.hidden_size] + out[:, -1, self.hidden_size:]             # proposed method sums the outputs involving adding the corresponding elements from both LSTM layers. 
        # out = self.dropout(out)
        out = torch.softmax(self.fc_1(out), dim=1)                                      # The sequence output from the Bi-LSTM is ultimately processed through Softmax to generate classification results.
                                
        return out

In [37]:
input_size = 11  # feature extracted data
hidden_size = 11 # the output dim is same as the input as the diagram shown in paper.
num_layers = 1   # it same just one layer in original paper. 
num_classes = 4
learning_rate = 1e-3

model = BiLSTMModel(input_size, hidden_size, num_layers, num_classes).to(device)

In [38]:
def train(dataloader, loss_fn, optimizer):
    total_acc, total_count, total_loss, = 0, 0, 0
    model.train()
    for series, label in dataloader:
        predicted_label = model(series)
        loss = loss_fn(predicted_label, label)
        # Back-propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            total_acc += (predicted_label.argmax(1) == label).sum().item()  # predict true
            total_count += label.size(0)
            total_loss += loss.item()*label.size(0)
    return total_loss/total_count, total_acc/total_count

def test(dataloader, loss_fn):
    model.eval()
    total_acc, total_count, total_loss, = 0, 0, 0

    with torch.no_grad():
        for series, label in dataloader:
            predicted_label = model(series)
            loss = loss_fn(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            total_loss += loss.item()*label.size(0)
    return total_loss/total_count, total_acc/total_count

def fit(epochs, train_dl, test_dl, loss_fn, optimizer):
    train_loss = [] 
    train_acc = []
    test_loss = []
    test_acc = []

    for epoch in range(epochs):
        
        epoch_loss, epoch_acc = train(train_dl, loss_fn, optimizer)
        epoch_test_loss, epoch_test_acc = test(test_dl, loss_fn)
        train_loss.append(epoch_loss)
        train_acc.append(epoch_acc)
        test_loss.append(epoch_test_loss)
        test_acc.append(epoch_test_acc)

        # template = ("epoch:{:2d}, train_loss: {:.5f}, train_acc: {:.1f}% ," 
        #             "test_loss: {:.5f}, test_acc: {:.1f}%")
        # if epoch%10 ==0:
        #     print(template.format(
        #         epoch, epoch_loss, epoch_acc*100, epoch_test_loss, epoch_test_acc*100))
    
    return train_loss, test_loss, train_acc, test_acc

In [42]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), betas=(0.5, 0.5), lr=0.001)

EPOCHS = 100
train_loss, test_loss, train_acc, test_acc = fit(EPOCHS, train_loader, test_loader, loss_fn, optimizer)

'''(100-200 epoch)
epoch: 0, train_loss: 0.89473, train_acc: 84.9% ,test_loss: 0.95245, test_acc: 79.4%
epoch:10, train_loss: 0.89173, train_acc: 85.2% ,test_loss: 0.94924, test_acc: 79.5%
epoch:20, train_loss: 0.88831, train_acc: 85.6% ,test_loss: 0.94837, test_acc: 79.6%
epoch:30, train_loss: 0.88404, train_acc: 85.9% ,test_loss: 0.96708, test_acc: 77.8%
epoch:40, train_loss: 0.88291, train_acc: 86.1% ,test_loss: 0.92994, test_acc: 81.4%
epoch:50, train_loss: 0.87719, train_acc: 86.7% ,test_loss: 0.92345, test_acc: 82.4%
epoch:60, train_loss: 0.87594, train_acc: 86.9% ,test_loss: 0.93170, test_acc: 81.2%
epoch:70, train_loss: 0.87481, train_acc: 86.9% ,test_loss: 0.91622, test_acc: 82.9%
epoch:80, train_loss: 0.87333, train_acc: 87.1% ,test_loss: 0.95129, test_acc: 78.9%
epoch:90, train_loss: 0.86845, train_acc: 87.5% ,test_loss: 0.93593, test_acc: 80.1%
'''

epoch: 0, train_loss: 0.89473, train_acc: 84.9% ,test_loss: 0.95245, test_acc: 79.4%
epoch:10, train_loss: 0.89173, train_acc: 85.2% ,test_loss: 0.94924, test_acc: 79.5%
epoch:20, train_loss: 0.88831, train_acc: 85.6% ,test_loss: 0.94837, test_acc: 79.6%
epoch:30, train_loss: 0.88404, train_acc: 85.9% ,test_loss: 0.96708, test_acc: 77.8%
epoch:40, train_loss: 0.88291, train_acc: 86.1% ,test_loss: 0.92994, test_acc: 81.4%
epoch:50, train_loss: 0.87719, train_acc: 86.7% ,test_loss: 0.92345, test_acc: 82.4%
epoch:60, train_loss: 0.87594, train_acc: 86.9% ,test_loss: 0.93170, test_acc: 81.2%
epoch:70, train_loss: 0.87481, train_acc: 86.9% ,test_loss: 0.91622, test_acc: 82.9%
epoch:80, train_loss: 0.87333, train_acc: 87.1% ,test_loss: 0.95129, test_acc: 78.9%
epoch:90, train_loss: 0.86845, train_acc: 87.5% ,test_loss: 0.93593, test_acc: 80.1%


In [43]:
# Load the data
error = {}
for snr in range(0,25,5):

    train_data = pd.read_csv(f'train_{snr}db.csv')
    test_data = pd.read_csv(f'test_{snr}db.csv')

    train_dataset = TimeSeriesDataset(train_data)
    test_dataset = TimeSeriesDataset(test_data)

    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

    model = BiLSTMModel(input_size, hidden_size, num_layers, num_classes).to(device)

    train_loss, test_loss, train_acc, test_acc = fit(EPOCHS, train_loader, test_loader, loss_fn, optimizer)

    error[str(snr)] = [train_loss, test_loss, train_acc, test_acc]

epoch: 0, train_loss: 1.38429, train_acc: 28.6% ,test_loss: 1.39237, test_acc: 18.2%
epoch:10, train_loss: 1.38429, train_acc: 28.6% ,test_loss: 1.39237, test_acc: 18.2%
epoch:20, train_loss: 1.38429, train_acc: 28.6% ,test_loss: 1.39237, test_acc: 18.2%
epoch:30, train_loss: 1.38429, train_acc: 28.6% ,test_loss: 1.39237, test_acc: 18.2%
epoch:40, train_loss: 1.38429, train_acc: 28.6% ,test_loss: 1.39237, test_acc: 18.2%
epoch:50, train_loss: 1.38429, train_acc: 28.6% ,test_loss: 1.39237, test_acc: 18.2%
epoch:60, train_loss: 1.38429, train_acc: 28.6% ,test_loss: 1.39237, test_acc: 18.2%
epoch:70, train_loss: 1.38429, train_acc: 28.6% ,test_loss: 1.39237, test_acc: 18.2%
epoch:80, train_loss: 1.38429, train_acc: 28.6% ,test_loss: 1.39237, test_acc: 18.2%
epoch:90, train_loss: 1.38429, train_acc: 28.6% ,test_loss: 1.39237, test_acc: 18.2%


FileNotFoundError: [Errno 2] No such file or directory: 'train_1db.csv'